In [5]:
from PyPDF2 import PdfReader
from pdfplumber import pdf
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer, util
import time
from gpt4all import GPT4All

from numba import jit, cuda
import numpy as np


In [6]:
llm = GPT4All("orca-mini-3b.ggmlv3.q4_0.bin")
# llm = GPT4All("D:\project\pdf_reader\pdf_reader\pdf_reader_v2\models\ggml-old-vic13b-q4_0.bin")
# llm = GPT4All("D:\project\pdf_reader\pdf_reader\pdf_reader_v2\models\ggml-gpt4all-j-v1.3-groovy.bin")

Found model file at  C:\\\\Users\\\\yasha\\\\.cache\\\\gpt4all\\orca-mini-3b.ggmlv3.q4_0.bin


## Querying the pdf

In [40]:
question = "car is not starting"

# Load and split the PDF

In [15]:
pdf_path = "D:/project/pdf_reader/pdf_reader/pdf_reader_v2/document/AltoK10_Owner's_Manual.pdf"
pdf_loader = PdfReader(open(pdf_path, "rb"))
pdf_text = ""

In [16]:
for page_num in range(len(pdf_loader.pages)):
    pdf_page = pdf_loader.pages[page_num]
    pdf_text += pdf_page.extract_text()

# Split the text using RecursiveCharacterTextSplitter

In [46]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2048,
    chunk_overlap=64
)
split_texts = text_splitter.split_text(pdf_text)

# Join the split text into paragraphs or sentences

In [47]:
paragraphs = []
for text in split_texts:
    paragraphs.extend(text.split('\n')) 

# Initialize the SentenceTransformer model (Hugging Face)

In [48]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = SentenceTransformer(model_name)

# Query each paragraph and get responses

In [49]:
responses = []
for paragraph in paragraphs:
    response = embeddings.encode([paragraph], convert_to_tensor=True)
    responses.append((paragraph, response))

# Now, you can choose the most relevant response for the given query

In [50]:
query_embedding = embeddings.encode([question], convert_to_tensor=True)
best_response = None
best_similarity = -1.0

In [51]:
# count = 0
answers = []

for paragraph, response in responses:
    
    similarity = util.pytorch_cos_sim(query_embedding, response).item()
    
    if similarity >= 0.6:
        
        # count += 1
        
        answers.append(paragraph)
    
    # if count > 5:
        
    #     break

answer = "\n".join(answers)

In [52]:
print (answers)

['engine does not start after severalattempts, check the fuel and igni-', 'If the engine still does not start, try holding']


In [53]:
prompt_message = answer + "\n" + question

final_response = llm.generate(prompt=prompt_message)

In [45]:
print (final_response)

. Check the fuel level and ignition switch. If the fuel level is low, add more fuel to the tank. If the ignition switch is not working, replace it. 2. If the engine still does not start, check the starter motor. If the starter motor is faulty, you may need to replace it. 3. If the engine still does not start, try starting the engine with a jump start or by hand. If this does not work, the engine may be damaged and require further repair.
